# Import Library

In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
import os

# Convert to TF Lite

In [ ]:
# --- Konfigurasi ---
# Jalur ke model hasil fine-tuning yang sudah Anda simpan
MODEL_PATH = '../models/klasifikasi_sampah_final_v1.h5'

# Jalur untuk menyimpan model TensorFlow Lite yang sudah dikuantisasi
QUANTIZED_MODEL_PATH = '../models/klasifikasi_sampah_tflite_v1.tflite'

In [ ]:
# Ukuran gambar yang digunakan model (harus sama dengan yang digunakan saat training)
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
TARGET_IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)

In [ ]:
# --- Bagian 1: Muat Model Keras yang Sudah Dilatih ---
print(f"\nMemuat model Keras dari: {MODEL_PATH}")
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print("Model Keras berhasil dimuat.")
    model.summary()
except Exception as e:
    print(f"Error: Gagal memuat model Keras dari '{MODEL_PATH}'. Pastikan jalur model benar dan file tidak rusak.")
    print(e)
    exit() # Hentikan eksekusi jika model tidak dapat dimuat


Memuat model Keras dari: ../models/klasifikasi_sampah_final_v1.h5


Model Keras berhasil dimuat.


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,436 (9.87 MB)

 Trainable params: 2,518,210 (9.61 MB)

 Non-trainable params: 68,224 (266.50 KB)

 Optimizer params: 2 (12.00 B)

In [ ]:
# --- Bagian 2: Lakukan Kuantisasi Menggunakan TensorFlow Lite Converter ---

# Metode Kuantisasi: Post-training float16 quantization
# Mengurangi ukuran model dengan mengubah bobot menjadi float16
print("\nMelakukan kuantisasi ke float16...")
converter_float16 = tf.lite.TFLiteConverter.from_keras_model(model)
converter_float16.optimizations = [tf.lite.Optimize.DEFAULT]
converter_float16.target_spec.supported_types = [tf.float16]

try:
    tflite_model_float16 = converter_float16.convert()
    print("Kuantisasi ke float16 berhasil.")
except Exception as e:
    print(f"Error saat kuantisasi ke float16: {e}")
    # Anda bisa mencoba kuantisasi integer jika float16 gagal, atau debug errornya.



Melakukan kuantisasi ke float16...
INFO:tensorflow:Assets written to: C:\Users\RAFI\AppData\Local\Temp\tmpu_bui3vb\assets


INFO:tensorflow:Assets written to: C:\Users\RAFI\AppData\Local\Temp\tmpu_bui3vb\assets


Saved artifact at 'C:\Users\RAFI\AppData\Local\Temp\tmpu_bui3vb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_4')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  1786490229136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490229712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490231440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490231056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490229904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490231632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490232208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490232400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490230288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1786490230864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  17864

In [ ]:
# --- Bagian 3: Simpan Model yang Sudah Dikuantisasi ---

# Tentukan jalur penyimpanan untuk model yang dikuantisasi

print(f"\nMenyimpan model TensorFlow Lite (float16) ke: {QUANTIZED_MODEL_PATH}")
try:
    with open(QUANTIZED_MODEL_PATH, 'wb') as f:
        f.write(tflite_model_float16)
    print(f"Model TensorFlow Lite (float16) berhasil disimpan. Ukuran: {os.path.getsize(QUANTIZED_MODEL_PATH) / (1024 * 1024):.2f} MB")
except Exception as e:
    print(f"Error saat menyimpan model float16: {e}")


Menyimpan model TensorFlow Lite (float16) ke: ../models/klasifikasi_sampah_tflite_v1.tflite
Model TensorFlow Lite (float16) berhasil disimpan. Ukuran: 4.88 MB


# Convert to Tensorflow Js

In [ ]:
# --- Konfigurasi Jalur Model ---
# Pastikan jalur ini sesuai dengan lokasi model Anda
MODEL_H5_PATH = '../models/klasifikasi_sampah_ft_v1.h5' 

# Direktori output untuk model TensorFlow.js
# Ini akan membuat folder yang berisi file model.json dan shard weights
TFJS_MODEL_DIR = '../models/Tensorflow.js'

# Pastikan direktori output ada
os.makedirs(TFJS_MODEL_DIR, exist_ok=True)

In [ ]:
# Pastikan direktori output ada
# Ini akan membuat folder jika belum ada. `exist_ok=True` mencegah error jika folder sudah ada.
os.makedirs(TFJS_MODEL_DIR, exist_ok=True)
print(f"Memastikan direktori output '{TFJS_MODEL_DIR}' tersedia.")

# --- Bagian 2: Muat Model Keras (.h5) ---
print(f"\nMemuat model Keras dari: {MODEL_H5_PATH}")
try:
    model = tf.keras.models.load_model(MODEL_H5_PATH)
    print("Model Keras berhasil dimuat.")
    model.summary()
except Exception as e:
    print(f"Error: Gagal memuat model Keras dari '{MODEL_H5_PATH}'.")
    print(f"Pastikan jalur model di Google Drive benar dan file tidak rusak. Detail error: {e}")
    exit() # Hentikan eksekusi jika model tidak dapat dimuat

In [ ]:
# --- Bagian 3: Konversi Model ke Format TensorFlow.js ---
print(f"\nMengonversi model ke format TensorFlow.js di: {TFJS_MODEL_DIR}")
try:
    # Untuk model Keras (.h5), gunakan tfjs.converters.save_keras_model
    tfjs.converters.save_keras_model(model, TFJS_MODEL_DIR)
    print("Konversi ke TensorFlow.js berhasil!")
    print("\nFile yang dihasilkan di Google Drive Anda:")
    # Menampilkan daftar file yang dihasilkan untuk konfirmasi
    for root, dirs, files in os.walk(TFJS_MODEL_DIR):
        for name in files:
            print(os.path.join(root, name))
except Exception as e:
    print(f"Error saat mengonversi model ke TensorFlow.js: {e}")
    print("Pastikan Anda telah menginstal `tensorflowjs` (`pip install tensorflowjs`)")
    print("Dan pastikan versi TensorFlow serta TensorFlow.js kompatibel.")
    exit()

print("\nModel TensorFlow.js siap untuk digunakan di aplikasi web.")


Memastikan direktori output '/content/drive/My Drive/IBM Project/Models' tersedia.

Memuat model Keras dari: /content/drive/My Drive/IBM Project/klasifikasi_sampah_final_v1.h5


Model Keras berhasil dimuat.


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,436 (9.87 MB)

 Trainable params: 2,518,210 (9.61 MB)

 Non-trainable params: 68,224 (266.50 KB)

 Optimizer params: 2 (12.00 B)


Mengonversi model ke format TensorFlow.js di: /content/drive/My Drive/IBM Project/Models
failed to lookup keras version from the file,
    this is likely a weight only file
Konversi ke TensorFlow.js berhasil!

File yang dihasilkan di Google Drive Anda:
/content/drive/My Drive/IBM Project/Models/group1-shard1of3.bin
/content/drive/My Drive/IBM Project/Models/group1-shard2of3.bin
/content/drive/My Drive/IBM Project/Models/group1-shard3of3.bin
/content/drive/My Drive/IBM Project/Models/model.json

Model TensorFlow.js siap untuk digunakan di aplikasi web.
